# Title: Block VQE with Runtime

In this file we will generate the blocks from the Hamiltoniain and run a previously saved runtime program.  

# Generating the blocks

For now I will generate the blocks from a matrix representation of the Hamiltonian.  This can be optimized later on using binary numbers and masks.  

We will explore this Hamiltonian:
$$ 
    H =  \sum_{i} \left(x X_{i}X_{i+1} + y Y_{i} Y_{i+1} + z Z_{i} Z_{i+1}\right) 
$$
breaking it up into two piecies, one which will be solved classically and the other which will be solved via VQE.

\begin{equation}
\begin{split}
H_{c,c'}=& T_{c,c'} + \delta_{cc'}\sum_{i=0}^{N_q} \left(x X_{i}X_{i+1} + y Y_{i} Y_{i+1}  + z Z_{i} Z_{i+1}\right) 
\\
&+ \left(B^{1x}_{c,c'}X_{0} + B^{1y}_{c,c'} Y_{0} + B^{1z}_{c,c'} Z_{0} \right)
\end{split}
\end{equation}
Notice that I have shifted the chaing so that the quantum part now starts at $i=0$.  The classical part contains the following paramters
\begin{equation}
\begin{split}
& T_{c,c'} =  <c'|\sum_{i=0}^{N_c-2}\left(x X_{i}X_{i+1} + y Y_{i} Y_{i+1} + z Z_{i} Z_{i+1}\right)|c> 
\\
& B1^x_{c,c'} =  x<c'|X_{N_c-1}|c>
\\
&B1^y_{c,c'} = y<c'|Y_{N_c-1}|c> 
\\
&B1^z_{c,c'} = z<c'|Y_{N_c-1}|c> 
\end{split}
\end{equation}

In [28]:
import qiskit.quantum_info as qi

def X(i,N):
    label = ['I' for i in range(N)]
    label[i] = 'X'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Y(i,N):
    label = ['I' for i in range(N)]
    label[i] = 'Y'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Z(i,N):
    label = ['I' for i in range(N)]
    label[i] = 'Z'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

In [29]:
# A function to print the state given the numerical represenations
def bi(num,N):
    bi = bin(num)
    out = []
    Sdiff = N - len(bi) + 2
    for i in range(0,Sdiff):
        out.append(0)
    for i in range(2,len(bi)):
        out.append(int(bi[i]))
    return out

# A function which retruns the numerical representation of states given N
def states(N):
    out = [i for i in range(0,2**N)]
    return out

# A function to print the basis vectors given the number of qubits N
def vecs(N):
    out = []
    for i in range(2**N):
        v = [0 for i in range(2**N)]
        v[i] = 1
        out.append(v)
    return out

In [30]:
import numpy as np

def Mdot(Ol):
    out = Ol[0]
    for i in range(1,len(Ol)):
        out = np.dot(Ol[i],out)
    return out

def bkt(y1,O,y2):
    return Mdot([np.conjugate(y1),O,y2])

In [31]:
def H(x,y,z,N):
    h = 0*X(0,N)
    for i in range(0,N-1):
        h += x*Mdot([X(i,N),X(i+1,N)]) + y*Mdot([Y(i,N),Y(i+1,N)]) + z*Mdot([Z(i,N),Z(i+1,N)])
    return h
        
    
import pandas as pd

In [32]:
def T(x,y,z,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    for i in range(0,Nc-1):
        O = x*Mdot([X(i,Nc),X(i+1,Nc)]) + y*Mdot([Y(i,Nc),Y(i+1,Nc)]) + z*Mdot([Z(i,Nc),Z(i+1,Nc)])
        out += bkt(v[cc],O,v[c])
    return out

def Bx(x,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    O = X(Nc-1,Nc)
    out += bkt(v[cc],x*O,v[c])
    return out

def By(y,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    O = Y(Nc-1,Nc)
    out += bkt(v[cc],y*O,v[c])
    return out

def Bz(z,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    O = Z(Nc-1,Nc)
    out += bkt(v[cc],z*O,v[c])
    return out

In [33]:
from qiskit.opflow.primitive_ops import MatrixOp
from qiskit import quantum_info as qi

def H_quantum(x,y,z,c,cc,Nc,Nq):
    tcc = T(x,y,z,c,cc,Nc)
    Iq = np.identity(2**Nq)
    h = [MatrixOp(tcc*Iq).to_pauli_op()]
    h.append(MatrixOp(Bx(x,c,cc,Nc)*X(0,Nq)).to_pauli_op())
    h.append(MatrixOp(By(y,c,cc,Nc)*Y(0,Nq)).to_pauli_op())
    h.append(MatrixOp(Bz(z,c,cc,Nc)*Z(0,Nq)).to_pauli_op())
    if c == cc:
        for i in range(0,Nq-1):
            h.append(MatrixOp(x*Mdot([X(i,Nq),X(i+1,Nq)])).to_pauli_op())
            h.append(MatrixOp(y*Mdot([Y(i,Nq),Y(i+1,Nq)])).to_pauli_op())
            h.append(MatrixOp(z*Mdot([Z(i,Nq),Z(i+1,Nq)])).to_pauli_op())
    return h

In [34]:
from qiskit.opflow import SummedOp, PauliOp

def Blocks(x,y,z,Nc,Nq):
    bks = {}
    Mc = len(states(Nc))
    for c in range(0,Mc):
        for cc in range(0,Mc):
            Op = H_quantum(x,y,z,c,cc,Nc,Nq)
            bks[str(c) + ',' + str(cc)] =  Op
    return bks

In [35]:
#Blocks(1,2,3,2,3)

Compare the blocks to the original Hamiltonian

In [36]:
Nc = 2
Nq = 3
x = 1
y = 2
z = 3
blocks = Blocks(x,y,z,Nc,Nq)

keys = list(blocks.keys())

bL = len(states(Nc))
size = 2**Nq
H_complete = np.array([[0 for i in range(size*bL)] for j in range(size*bL)])

for bi in range(bL):
    for bj in range(bi,bL):
        key = str(bi) + ',' + str(bj)
        if key  in keys:
            hl = blocks[key]
            block = np.sum(hl).to_matrix()
        else:
            block = np.array([[0.0 for i in range(size)] for j in range(size)])
        for i in range(size):
            for j in range(size):
                H_complete[i + bi*size, j + bj*size] = block[i,j]
                H_complete[i + bj*size, j + bi*size] = block[j,i]

<ipython-input-36-f43942b05d04>:24: ComplexWarning: Casting complex values to real discards the imaginary part
  H_complete[i + bi*size, j + bj*size] = block[i,j]
<ipython-input-36-f43942b05d04>:25: ComplexWarning: Casting complex values to real discards the imaginary part
  H_complete[i + bj*size, j + bi*size] = block[j,i]


In [37]:
np.amax(np.abs(H_complete - np.real(H(x,y,z,Nc+Nq))))

0.0

# Running the program

## Build the initial anzats angles

In [38]:
bL = 2**Nc
alpha = 1/np.sqrt(bL)*np.array([1 for b in range(bL)])
#number of blocks
T = 2
phi = []
for b in range(bL):
    phi_b = []
    for ti in range(T):
        phi_t = []
        for q in range(Nq):
            phi_t.append(0)
        phi_b.append(phi_t)
    phi.append(phi_b)

## Importing the runtime program

In [119]:
import os
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')  # Substitute with your provider.


ibmqfactory.load_account:WARNING:2021-11-02 10:39:54,389: Credentials are already in use. The existing account in the session will be replaced.


In [190]:
program_id = 'block-vqe'
my_program = provider.runtime.program(program_id)

#print(my_program)

## Run the program

### Simulation

In [191]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}")

In [195]:
backend = provider.backend.ibmq_qasm_simulator
options = {'backend_name': backend.name()}
inputs = {"k_max": 2000, "phi": phi, "alpha": alpha, "blocks":blocks, "seeded":True, "method":"simple"}
job = provider.runtime.run(program_id, options=options, inputs=inputs, callback=interim_result_callback)

In [196]:
print(f"job id: {job.job_id()}")
result = job.result()
#print(result)

job id: c60ls7t0p3h5hl49umk0
interim result: {'Starting program with k_max': 2000}
interim result: {'seeding is on': True}
interim result: {'starting iteration': 0}
interim result: {'k': 0, 'E_A': (6.394769974311853+0j)}
interim result: {'k': 0, 'E_B': (2.830183628909674+0j)}
interim result: {'k': 0, 'E_f': 5.258741406612572}
interim result: {'starting iteration': 1}
interim result: {'k': 1, 'E_A': (3.4707413889781233+0j)}
interim result: {'k': 1, 'E_B': (8.068810435861572+0j)}
interim result: {'k': 1, 'E_f': 4.477584957491599}
interim result: {'starting iteration': 2}
interim result: {'k': 2, 'E_A': (2.9701641465724364+0j)}
interim result: {'k': 2, 'E_B': (5.713708436891429+0j)}
interim result: {'k': 2, 'E_f': 4.180102714107959}
interim result: {'starting iteration': 3}
interim result: {'k': 3, 'E_A': (3.496420674330254+0j)}
interim result: {'k': 3, 'E_B': (1.9265897725870582+0j)}
interim result: {'k': 3, 'E_f': 3.8590587539580445}
interim result: {'starting iteration': 4}
interim res

interim result: {'k': 38, 'E_B': (-9.076959790152555+0j)}
interim result: {'k': 38, 'E_f': -9.240387119483533}
interim result: {'starting iteration': 39}
interim result: {'k': 39, 'E_A': (-5.7887316457552425+0j)}
interim result: {'k': 39, 'E_B': (-7.217413078727966+0j)}
interim result: {'k': 39, 'E_f': -9.689168155032512}
interim result: {'starting iteration': 40}
interim result: {'k': 40, 'E_A': (-4.060374144811577+0j)}
interim result: {'k': 40, 'E_B': (-10.216829765779215+0j)}
interim result: {'k': 40, 'E_f': -10.65116763013507}
interim result: {'starting iteration': 41}
interim result: {'k': 41, 'E_A': (-7.202187666524278+0j)}
interim result: {'k': 41, 'E_B': (-8.755434035024548+0j)}
interim result: {'k': 41, 'E_f': -10.708795712745827}
interim result: {'starting iteration': 42}
interim result: {'k': 42, 'E_A': (-6.454882351973064+0j)}
interim result: {'k': 42, 'E_B': (-7.8988006824288+0j)}
interim result: {'k': 42, 'E_f': -10.541750328461967}
interim result: {'starting iteration': 

interim result: {'k': 77, 'E_A': (-8.278422375557343+0j)}
interim result: {'k': 77, 'E_B': (-6.85338250648559+0j)}
interim result: {'k': 77, 'E_f': -10.398004755442308}
interim result: {'starting iteration': 78}
interim result: {'k': 78, 'E_A': (-5.637037628205738+0j)}
interim result: {'k': 78, 'E_B': (-8.571745436001741+0j)}
interim result: {'k': 78, 'E_f': -10.146193352299157}
interim result: {'starting iteration': 79}
interim result: {'k': 79, 'E_A': (-9.040983197601284+0j)}
interim result: {'k': 79, 'E_B': (-5.879488319057121+0j)}
interim result: {'k': 79, 'E_f': -10.073253237703787}
interim result: {'starting iteration': 80}
interim result: {'k': 80, 'E_A': (-7.0432526846292856+0j)}
interim result: {'k': 80, 'E_B': (-8.028469127896336+0j)}
interim result: {'k': 80, 'E_f': -9.950607360114628}
interim result: {'starting iteration': 81}
interim result: {'k': 81, 'E_A': (-7.561179449857862+0j)}
interim result: {'k': 81, 'E_B': (-9.072675047991014+0j)}
interim result: {'k': 81, 'E_f': 

interim result: {'k': 115, 'E_f': -11.199604463994067}
interim result: {'starting iteration': 116}
interim result: {'k': 116, 'E_A': (-9.107294316409119+0j)}
interim result: {'k': 116, 'E_B': (-8.871011827687019+0j)}
interim result: {'k': 116, 'E_f': -11.234838940430956}
interim result: {'starting iteration': 117}
interim result: {'k': 117, 'E_A': (-8.651329385433453+0j)}
interim result: {'k': 117, 'E_B': (-5.495297427065793+0j)}
interim result: {'k': 117, 'E_f': -10.666003618944275}
interim result: {'starting iteration': 118}
interim result: {'k': 118, 'E_A': (-8.64445179177624+0j)}
interim result: {'k': 118, 'E_B': (-9.015624353130688+0j)}
interim result: {'k': 118, 'E_f': -10.712260311979508}
interim result: {'starting iteration': 119}
interim result: {'k': 119, 'E_A': (-8.748388949191419+0j)}
interim result: {'k': 119, 'E_B': (-6.9009667759658715+0j)}
interim result: {'k': 119, 'E_f': -10.503944480373521}
interim result: {'starting iteration': 120}
interim result: {'k': 120, 'E_A':

interim result: {'k': 153, 'E_f': -11.185820067359836}
interim result: {'starting iteration': 154}
interim result: {'k': 154, 'E_A': (-8.571490211812279+0j)}
interim result: {'k': 154, 'E_B': (-9.74231036074647+0j)}
interim result: {'k': 154, 'E_f': -11.154368103708354}
interim result: {'starting iteration': 155}
interim result: {'k': 155, 'E_A': (-8.959038400668302+0j)}
interim result: {'k': 155, 'E_B': (-11.512239250469712+0j)}
interim result: {'k': 155, 'E_f': -11.55680429256632}
interim result: {'starting iteration': 156}
interim result: {'k': 156, 'E_A': (-8.34058945773728+0j)}
interim result: {'k': 156, 'E_B': (-10.777467433454369+0j)}
interim result: {'k': 156, 'E_f': -11.546292101644942}
interim result: {'starting iteration': 157}
interim result: {'k': 157, 'E_A': (-9.967849703291211+0j)}
interim result: {'k': 157, 'E_B': (-9.7567714554365+0j)}
interim result: {'k': 157, 'E_f': -11.532617948569712}
interim result: {'starting iteration': 158}
interim result: {'k': 158, 'E_A': (-

...

interim result: {'k': 1835, 'E_A': (-12.461322105048714+0j)}
interim result: {'k': 1835, 'E_B': (-13.880465953981313+0j)}
interim result: {'k': 1835, 'E_f': -14.561898633888761}
interim result: {'starting iteration': 1836}
interim result: {'k': 1836, 'E_A': (-13.22453382986965+0j)}
interim result: {'k': 1836, 'E_B': (-13.230972686439657+0j)}
interim result: {'k': 1836, 'E_f': -14.562355823579328}
interim result: {'starting iteration': 1837}
interim result: {'k': 1837, 'E_A': (-12.998446020835871+0j)}
interim result: {'k': 1837, 'E_B': (-13.355110522859759+0j)}
interim result: {'k': 1837, 'E_f': -14.54098052240254}
interim result: {'starting iteration': 1838}
interim result: {'k': 1838, 'E_A': (-12.671906027104217+0j)}
interim result: {'k': 1838, 'E_B': (-13.4493687349036+0j)}
interim result: {'k': 1838, 'E_f': -14.52247955860913}
interim result: {'starting iteration': 1839}
interim result: {'k': 1839, 'E_A': (-12.69599295996755+0j)}
interim result: {'k': 1839, 'E_B': (-12.3595262178256

interim result: {'k': 1872, 'E_A': (-11.864892528886696+0j)}
interim result: {'k': 1872, 'E_B': (-13.824221839847858+0j)}
interim result: {'k': 1872, 'E_f': -14.657264567407534}
interim result: {'starting iteration': 1873}
interim result: {'k': 1873, 'E_A': (-13.695203195169844+0j)}
interim result: {'k': 1873, 'E_B': (-12.191052564645895+0j)}
interim result: {'k': 1873, 'E_f': -14.640454084228415}
interim result: {'starting iteration': 1874}
interim result: {'k': 1874, 'E_A': (-13.674434361808311+0j)}
interim result: {'k': 1874, 'E_B': (-13.028582292758383+0j)}
interim result: {'k': 1874, 'E_f': -14.618559446274709}
interim result: {'starting iteration': 1875}
interim result: {'k': 1875, 'E_A': (-12.64030096098225+0j)}
interim result: {'k': 1875, 'E_B': (-13.168294969561474+0j)}
interim result: {'k': 1875, 'E_f': -14.583590947337187}
interim result: {'starting iteration': 1876}
interim result: {'k': 1876, 'E_A': (-13.589820124233958+0j)}
interim result: {'k': 1876, 'E_B': (-13.37343594

interim result: {'k': 1909, 'E_A': (-12.951070308964301+0j)}
interim result: {'k': 1909, 'E_B': (-13.23993054787635+0j)}
interim result: {'k': 1909, 'E_f': -14.801082390863773}
interim result: {'starting iteration': 1910}
interim result: {'k': 1910, 'E_A': (-12.241704875510779+0j)}
interim result: {'k': 1910, 'E_B': (-14.034337923685186+0j)}
interim result: {'k': 1910, 'E_f': -14.804216246934269}
interim result: {'starting iteration': 1911}
interim result: {'k': 1911, 'E_A': (-13.190825592012+0j)}
interim result: {'k': 1911, 'E_B': (-13.219114311148285+0j)}
interim result: {'k': 1911, 'E_f': -14.804631052441511}
interim result: {'starting iteration': 1912}
interim result: {'k': 1912, 'E_A': (-12.796477824499592+0j)}
interim result: {'k': 1912, 'E_B': (-12.910538463172017+0j)}
interim result: {'k': 1912, 'E_f': -14.79991476483524}
interim result: {'starting iteration': 1913}
interim result: {'k': 1913, 'E_A': (-13.205521526585175+0j)}
interim result: {'k': 1913, 'E_B': (-12.436822006947

interim result: {'k': 1946, 'E_A': (-13.502152569709857+0j)}
interim result: {'k': 1946, 'E_B': (-13.541897036182416+0j)}
interim result: {'k': 1946, 'E_f': -14.86249063939901}
interim result: {'starting iteration': 1947}
interim result: {'k': 1947, 'E_A': (-13.252089526002125+0j)}
interim result: {'k': 1947, 'E_B': (-13.477168521000193+0j)}
interim result: {'k': 1947, 'E_f': -14.860006180100008}
interim result: {'starting iteration': 1948}
interim result: {'k': 1948, 'E_A': (-12.768708697223726+0j)}
interim result: {'k': 1948, 'E_B': (-12.904249971929383+0j)}
interim result: {'k': 1948, 'E_f': -14.859983448428823}
interim result: {'starting iteration': 1949}
interim result: {'k': 1949, 'E_A': (-12.22150743216004+0j)}
interim result: {'k': 1949, 'E_B': (-13.132268769707146+0j)}
interim result: {'k': 1949, 'E_f': -14.861793575212912}
interim result: {'starting iteration': 1950}
interim result: {'k': 1950, 'E_A': (-13.118258487365832+0j)}
interim result: {'k': 1950, 'E_B': (-13.466251478

interim result: {'k': 1983, 'E_A': (-13.729392576009943+0j)}
interim result: {'k': 1983, 'E_B': (-11.817374320555793+0j)}
interim result: {'k': 1983, 'E_f': -14.640554682396147}
interim result: {'starting iteration': 1984}
interim result: {'k': 1984, 'E_A': (-13.112902911884428+0j)}
interim result: {'k': 1984, 'E_B': (-13.53525777403724+0j)}
interim result: {'k': 1984, 'E_f': -14.678565933934387}
interim result: {'starting iteration': 1985}
interim result: {'k': 1985, 'E_A': (-12.596983224710199+0j)}
interim result: {'k': 1985, 'E_B': (-12.5473478162169+0j)}
interim result: {'k': 1985, 'E_f': -14.67860831674755}
interim result: {'starting iteration': 1986}
interim result: {'k': 1986, 'E_A': (-12.340148664082346+0j)}
interim result: {'k': 1986, 'E_B': (-13.597737215364758+0j)}
interim result: {'k': 1986, 'E_f': -14.731239120495488}
interim result: {'starting iteration': 1987}
interim result: {'k': 1987, 'E_A': (-13.020707995412101+0j)}
interim result: {'k': 1987, 'E_B': (-12.92251303546

In [197]:
e,ys = np.linalg.eig(H_complete)

In [200]:
import matplotlib.pyplot as plt

E0 = [np.sort(e)[0] for i in range(len(result))]

El = [result[i]['E'] for i in range(len(result))]
tl = [i for i in range(len(El))]

plt.plot(tl,El)
plt.plot(tl,E0)

### On Chip

In [18]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}")

In [19]:
backend = provider.get_backend('ibmq_bogota')
options = {'backend_name': backend.name()}
inputs = {"k_max": 3, "phi": phi, "alpha": alpha, "blocks":blocks}
job = provider.runtime.run(program_id, options=options, inputs=inputs, callback=interim_result_callback)

In [ ]:
print(f"job id: {job.job_id()}")
result = job.result()

job id: c5skrdbrrcfjv06gq9p0


In [ ]:
result[2]['E']